In [1]:
import time

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.subplots

import numpy as np
import matplotlib.pyplot as plt
import dpx_control_hw as dch

In [2]:
def get_color(c, N, idx):
    import matplotlib as mpl
    cmap = mpl.cm.get_cmap(c)
    norm = mpl.colors.Normalize(vmin=0.0, vmax=N - 1)
    return cmap(norm(idx))

## Settings

In [3]:
CONFIG = '../config.conf'

## Find port and connect

In [4]:
port = dch.find_port()
if port is None:
    port = '/dev/ttyACM0'

dpx = dch.Dosepix(
    port_name=port,
    config_fn=CONFIG,
    thl_calib_fn=None,
    params_fn=None,
    bin_edges_fn=None
)

## ToT-measurement generator

In [5]:
tot_gen = dpx.dpm.measure_tot(
    frame_time=0,
    save_frames=None,
    out_dir=None,
    meas_time=None,
    make_hist=True,
    use_gui=True
)

## Create dash-app

In [6]:
CUMULATIVE = True
NORM = True
READ_ADC = False

In [7]:
interval = 100
update_interval = 15 # in seconds
start_time = time.time()
global last_time
last_time = time.time()
meas_times = []

if CUMULATIVE:
    plt_data = []
else:
    plt_data = {'bins': np.arange(4095), 'hist': np.zeros(4095)}

if READ_ADC:
    dpx.dpm.select_adc(analog_out='v_tha')
    adc_data = {'time': [], 'bias': [], 'dpx': []}

app = JupyterDash(__name__, update_title=None)
app.layout = html.Div(
    [
        dcc.Graph(id='live-update-graph', style={'height': '90vh'}),
        dcc.Interval(
            id="interval",
            interval=interval,
            n_intervals=0
        ),
        html.Div(children=[
            html.Label('ToT-range'),
            dcc.Slider(
                id='tot_slider',
                min=0,
                max=800,
                value=400
            )
        ])
    ]
)

@app.callback(
    Output('live-update-graph', 'figure'),
    [Input('interval', 'n_intervals')],
    Input('tot_slider', 'value')
)
def update_data(n, tot_slider_value):
    global last_time
    if CUMULATIVE:
        # if not (int(start_time - time.time()) % update_interval):
        if (time.time() - last_time) > update_interval:
            try:
                plt_data.append({
                    'bins': np.arange(4095),
                    'hist': np.sum(next(tot_gen), axis=0)
                })
                dpx.dpf.data_reset()
                meas_times.append( int(time.time() - start_time) )
            except ValueError:
                pass
            last_time = time.time()
        else:
            # Update measurement nevertheless
            try:
                next(tot_gen)
            except:
                pass
    else:
        try:
            plt_data['hist'] = np.sum(next(tot_gen), axis=0)
        except ValueError:
            pass
        
    if READ_ADC:
        try:
            adc_data['bias'].append( int(dpx.dpf.read_bias(), 16) )
            adc_data['dpx'].append( int(dpx.dpf.read_adc(), 16) )
            adc_data['time'].append( time.time() - start_time )
        except ValueError:
            pass

    if READ_ADC:
        fig = plotly.subplots.make_subplots(3, 1, vertical_spacing=0.05)
    else:
        fig = plotly.subplots.make_subplots(vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 10, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    if CUMULATIVE:
        for idx, trace in enumerate(plt_data):
            color = get_color('viridis', len(plt_data), idx)
            color = '#' + ''.join(['%02x' % (int(color_elm * 256) - 1) for color_elm in color[:-1]])

            y = trace['hist'][:tot_slider_value]
            if NORM:
                y = y / np.max(trace['hist'][:tot_slider_value])
            fig.add_trace({
                'x': trace['bins'][:tot_slider_value],
                'y': y,
                'name': '%d s' % meas_times[idx],
                'mode': 'lines+markers',
                'type': 'scatter',
                'line': dict(color=color)
            }, 1, 1)
    else:
        fig.add_trace({
            'x': plt_data['bins'][:tot_slider_value],
            'y': plt_data['hist'][:tot_slider_value],
            'name': 'ToT',
            'mode': 'lines+markers',
            'type': 'scatter'
        }, 1, 1)
        
    if READ_ADC:
        fig.add_trace({
            'x': adc_data['time'],
            'y': adc_data['dpx'],
            'name': 'DPX ADC',
            'mode': 'lines+markers',
            'type': 'scatter',
            'line': dict(color='red')
        }, 2, 1)
        fig.add_trace({
            'x': adc_data['time'],
            'y': adc_data['bias'],
            'name': 'Bias Voltage',
            'mode': 'lines+markers',
            'type': 'scatter',
            'line': dict(color='blue')
        }, 3, 1)

    return fig

## Run server

In [8]:
app.run_server(debug=True, mode='external')

Dash app running on http://127.0.0.1:8050/


## Store to file

In [9]:
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('meas.json', 'w') as file:
    json.dump({'plt_data': plt_data, 'meas_times': meas_times}, file, cls=NumpyEncoder)

Starting ToT Measurement!
